In [1]:

import pandas as pd
import numpy as np
from selenium import webdriver
import time


In [3]:
races = pd.read_csv('race_data1.csv')

In [4]:
races.head()

,season,round,circuitRef,circuitId,raceId,latitude,longitude,country,location,url
0,1950,1,silverstone,9,833,52.0786,-1.01694,UK,Silverstone,http://en.wikipedia.org/wiki/1950_British_Gran...
1,1950,2,monaco,6,834,43.7347,7.42056,Monaco,Monte-Carlo,http://en.wikipedia.org/wiki/1950_Monaco_Grand...
2,1950,3,indianapolis,19,835,39.7950,-86.23470,USA,Indianapolis,http://en.wikipedia.org/wiki/1950_Indianapolis...
3,1950,4,bremgarten,66,836,46.9589,7.40194,Switzerland,Bern,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...
4,1950,5,spa,13,837,50.4372,5.97139,Belgium,Spa,http://en.wikipedia.org/wiki/1950_Belgian_Gran...


In [5]:
races.shape

(1026, 10)

In [6]:
weather = races.iloc[:,[0,1,2]]

In [7]:
weather.head()

,season,round,circuitRef
0,1950,1,silverstone
1,1950,2,monaco
2,1950,3,indianapolis
3,1950,4,bremgarten
4,1950,5,spa


In [8]:
info = []

for link in races.url:
    try:
        df = pd.read_html(link)[0]
        if 'Weather' in list(df.iloc[:,0]):
            n = list(df.iloc[:,0]).index('Weather')
            info.append(df.iloc[n,1])
        else:
            df = pd.read_html(link)[1]
            if 'Weather' in list(df.iloc[:,0]):
                n = list(df.iloc[:,0]).index('Weather')
                info.append(df.iloc[n,1])
            else:
                df = pd.read_html(link)[2]
                if 'Weather' in list(df.iloc[:,0]):
                    n = list(df.iloc[:,0]).index('Weather')
                    info.append(df.iloc[n,1])
                else:
                    df = pd.read_html(link)[3]
                    if 'Weather' in list(df.iloc[:,0]):
                        n = list(df.iloc[:,0]).index('Weather')
                        info.append(df.iloc[n,1])
                    else:
                        driver = webdriver.Chrome()
                        driver.get(link)

                        # click language button
                        button = driver.find_element_by_link_text('Italiano')
                        button.click()
                        
                        clima = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[9]/td').text
                        info.append(clima) 
                                
    except:
        info.append('not found')

In [9]:
len(info)

1026

In [10]:
weather['weather'] = info

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
weather.head()

,season,round,circuitRef,weather
0,1950,1,silverstone,"Sunny, mild, dry."
1,1950,2,monaco,not found
2,1950,3,indianapolis,Rainy
3,1950,4,bremgarten,"Warm, dry and sunny"
4,1950,5,spa,"Warm, dry and sunny"


In [12]:
weather.tail()

,season,round,circuitRef,weather
1021,2020,4,silverstone,not found
1022,2020,5,silverstone,not found
1023,2020,6,catalunya,not found
1024,2020,7,spa,not found
1025,2020,8,monza,not found


In [13]:

weather_dict = {'weather_warm': ['soleggiato', 'clear', 'warm', 'hot', 'sunny', 'fine', 'mild', 'sereno'],
               'weather_cold': ['cold', 'fresh', 'chilly', 'cool'],
               'weather_dry': ['dry', 'asciutto'],
               'weather_wet': ['showers', 'wet', 'rain', 'pioggia', 'damp', 'thunderstorms', 'rainy'],
               'weather_cloudy': ['overcast', 'nuvoloso', 'clouds', 'cloudy', 'grey', 'coperto']}

In [14]:

weather_df = pd.DataFrame(columns = weather_dict.keys())

In [15]:
for col in weather_df:
    weather_df[col] = weather['weather'].map(lambda x: 1 if any(i in weather_dict[col] for i in x.lower().split()) else 0)

In [16]:
weather_df.head()

,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,1,0
3,1,0,1,0,0
4,1,0,1,0,0


In [17]:
weather_info = pd.concat([weather, weather_df], axis = 1)

In [18]:
weather_info.shape

(1026, 9)

In [19]:
weather_info.head()

,season,round,circuitRef,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,1950,1,silverstone,"Sunny, mild, dry.",0,0,0,0,0
1,1950,2,monaco,not found,0,0,0,0,0
2,1950,3,indianapolis,Rainy,0,0,0,1,0
3,1950,4,bremgarten,"Warm, dry and sunny",1,0,1,0,0
4,1950,5,spa,"Warm, dry and sunny",1,0,1,0,0


In [20]:
weather_info.tail()

,season,round,circuitRef,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
1021,2020,4,silverstone,not found,0,0,0,0,0
1022,2020,5,silverstone,not found,0,0,0,0,0
1023,2020,6,catalunya,not found,0,0,0,0,0
1024,2020,7,spa,not found,0,0,0,0,0
1025,2020,8,monza,not found,0,0,0,0,0


In [21]:

weather_info.to_csv('weather1.csv', index= False)